### Langchain Agents的使用介绍
langchain 通过提供 `Agents` 拓展其与外部平台交互能力，`Agents`核心思想是使用语言模型（LLM）选择一系列要执行的动作并通过`Tool`和`Toolkits`执行。在`chain`的链式行为中，一系列动作是硬编码在代码中。而在`Agents`中，语言模型被用作推理引擎来确定要采取的动作及其顺序。它有以下几个关键概念：
- Agent: 利用 LLM 和 Prompt设计来决定下一步该采取什么行动。
- Tools: Agent调用的函数，需要基于动作执行的权限。
- Toolkits: 一组用于实现特定目标的工具，通常包含3-5个工具。
- AgentExecutor: 通常是为 Agent 的 runtime， 调用agent并通过tool完成动作执行。

下面我们将逐步介绍如何利用 Agents 实现与搜索引擎和其他平台交互。

**Step 1. Load LLM models**  
Agent 依赖模型决策工具执行顺序，这里使用 gpt-3.5-turbo

In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

**Step 2. Defining Custom Tools**  
Tools 是 Agent 调用的功能。在这里有两个重要的考虑因素：

- 给予 Agent 访问相应工具的权限。
- 以对 Agent 最有帮助的方式描述工具作用。
如果没有这两点，试图构建的 Agent 将无法工作。如果没有正确配置工具权限，Agent 将永远无法完成动作执行。如果没有正确地描述工具作用，Agent 将不知如何正确使用它们。

LangChain提供了一个广泛的工具集，同时也可以轻松自定义（包括自定义描述）。完整的[工具列表](https://python.langchain.com/docs/integrations/tools/)。

这里为了展示细节，我们分别自定义工具和载入已有工具。

In [2]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

from langchain import SerpAPIWrapper
from langchain.agents import load_tools
from langchain.tools import BaseTool

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return SerpAPIWrapper().run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

tools = load_tools(["openweathermap-api"], llm)
tools.append(CustomSearchTool())

**Step 3. Defining Agent with Tools**  
langchain 提供了多种 Action agents 和 Plan-and-execute agents, 其中前者使用LLM来确定要采取的操作及其顺序。一个操作可以是使用工具并观察其输出，或向用户返回响应。后者主要通过Agent规划要做的事情，然后执行子任务来实现一个目标。详细的 Agent types可参考[文档](https://python.langchain.com/docs/modules/agents/agent_types/)。

这里我们使用 `Zero-shot ReAct`进行演示，它是 Action agents 的一种，使用ReAct框架，仅根据工具的描述来确定使用哪个工具。可以提供任意数量的工具。这也是官方文档中介绍为“最通用的 Agent ”。


In [3]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

**Step 4. Agent Execute**

In [4]:
agent.run(
    "What is langchain? What is the weather like in Beijing?"
)



> Entering new AgentExecutor chain...
I need to find information about "langchain" and the weather in Beijing.
Action: custom_search
Action Input: "langchain"
Observation: LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.
Thought:I have found information about "langchain". Now I need to find the weather in Beijing.
Action: OpenWeatherMap
Action Input: "Beijing"
Observation: In Beijing, the current weather is as follows:
Detailed status: scattered clouds
Wind speed: 2.97 m/s, direction: 192°
Humidity: 65%
Temperature: 
  - Current: 30.94°C
  - High: 30.94°C
  - Low: 30.94°C
  - Feels like: 36.01°C
Rain: {}
Heat index: None
Cloud cover: 45%
Thought:I now know the final answer.
Final Answer: LangChain is a framework designed to simplify t

'LangChain is a framework designed to simplify the creation of applications using large language models. The current weather in Beijing is scattered clouds with a temperature of 30.94°C.'

**Another agent: Self ask with search**  
Self ask with search 利用名为`Intermediate Answer`的工具实现查找与问题相关的事实答案。该工具实现类似于"自问自答"功能，提供了一个谷歌搜索API作为工具。

In [7]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Intermediate Answer",
        func=SerpAPIWrapper().run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: Carlos Alcaraz won the 2022 Men's single title while Poland's Iga Swiatek won the Women's single title defeating Tunisian's Ons Jabeur.
Follow up: Where is Carlos Alcaraz from?
Intermediate answer: El Palmar, Spain
So the final answer is: El Palmar, Spain

> Finished chain.


'El Palmar, Spain'